In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import dask.dataframe as dd
from dask.delayed import delayed

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('C:/Users/nrezaei/Documents/Ridehail_Fleetsize_Price_Scenarios.csv') 
# Get the paths from the DataFrame
paths = df['Outputs'][0:2].tolist() 

In [4]:
paths

['https://s3.us-east-2.amazonaws.com/beam-outputs/index.html#pilates-outputs/sfbay_1fleet_100price_100fleet_30pct_20230222/',
 'https://s3.us-east-2.amazonaws.com/beam-outputs/index.html#pilates-outputs/sfbay_baseline_30pct_20230218/']

In [5]:
# Iterate over the S3 paths in the dataframe
prefixs = []
for path in paths:
    prefix = path.replace('https://s3.us-east-2.amazonaws.com/beam-outputs/index.html#', '')
    prefix = prefix + 'inexus/'
    prefixs.append(prefix)

In [6]:
prefixs

['pilates-outputs/sfbay_1fleet_100price_100fleet_30pct_20230222/inexus/',
 'pilates-outputs/sfbay_baseline_30pct_20230218/inexus/']

In [7]:
s3 = boto3.client("s3")

bucket_name = 'beam-outputs'

# read all files into a list of dataframes
key_list = []

# navigate the folder and read the CSV files
for prefix in prefixs:
    # Use the S3 client to list all objects in the bucket and prefix
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    # extract the keys of the CSV files from the object list
    keys = [obj["Key"] for obj in objects["Contents"] if '_2019_' in obj['Key'] and obj["Key"].endswith(".csv.gz")]
    keys = ''.join(keys)
    key_list.append(keys)

CPU times: total: 109 ms
Wall time: 723 ms


In [9]:
key_list

['pilates-outputs/sfbay_1fleet_100price_100fleet_30pct_20230222/inexus/sfbay_rh_1fleet_fleets-1_2019__20230221.csv.gz',
 'pilates-outputs/sfbay_baseline_30pct_20230218/inexus/sfbay_baseline_default-1.0_2019__20230217.csv.gz']

In [10]:
cols_to_use = ['IDMerged', 'tripIndex', 'actStartTime', 'actEndTime','duration_travelling', 'cost_BEAM', 'actStartType', 
               'actEndType', 'duration_walking', 'duration_in_privateCar', 'duration_on_bike', 'duration_in_ridehail', 
              'distance_travelling', 'duration_in_transit', 'distance_walking','distance_bike','distance_ridehail', 
              'distance_privateCar', 'distance_transit', 'mode_choice_planned_BEAM','mode_choice_actual_BEAM',
              'vehicleIds', 'distance_mode_choice', 'replanning_status', 'reason', 'fuel_marginal','BlockGroupStart',
              'startX', 'startY', 'bgid_start', 'tractid_start', 'juris_name_start', 'county_name_start', 'mpo_start', 
               'BlockGroupEnd', 'endX', 'endY', 'bgid_end', 'tractid_end', 'juris_name_end', 'county_name_end', 'mpo_end', 
               'emission_marginal', 'duration_door_to_door', 'waitTime_no_replanning', 'waitTime_replanning', 'actPurpose', 
               'mode_choice_actual_6', 'mode_choice_actual_5', 'mode_choice_actual_4', 'trip_mode_AS_trips', 'logsum_trip_Potential_INEXUS',
               'age', 'income', 'hh_cars', 'TAZ_x', 'origin_x', 'destination_x', 'TAZ_y', 'home_taz', 'auto_ownership', 'home_is_urban', 'home_is_rural', 'DRIVEALONEFREE',
               'DRIVEALONEPAY', 'SHARED2FREE', 'SHARED2PAY', 'SHARED3FREE', 'SHARED3PAY', 'WALK', 'BIKE', 'WALK_LOC', 'WALK_LRF', 
               'WALK_EXP', 'WALK_HVY', 'WALK_COM', 'DRIVE_LOC', 'DRIVE_LRF', 'DRIVE_EXP', 'DRIVE_HVY', 'DRIVE_COM', 'TAXI',
               'TNC_SINGLE', 'TNC_SHARED', 'income_quartiles', 'income_deciles', 'value_of_time', 'primary_purpose_x',
              ] # Specify the columns to read

In [11]:
# define the filenames for each row in the DataFrame
filenames = []

for key in key_list:
    # split the key based on the `/` separator and take the last element
    filename = key.split('/')[-1].rstrip('.csv.gz')
    #filename = filename.split("_2019")[0]  # remove anything after the year 2019
    filenames.append(filename)

In [12]:
filenames

['sfbay_rh_1fleet_fleets-1_2019__20230221',
 'sfbay_baseline_default-1.0_2019__20230217']

In [17]:
custom_values = {}
    
for i, row in df[0:2].reset_index(drop=True).iterrows():
    # your code here.iterrows():
    # extract the custom values for the current row
    custom_value_1 = row['lever_position_price']
    custom_value_2 = row['lever_position_fltsz']
    custom_value_3 = row['lever_n_fleets']
    custom_value_4 = row['fleetsize_uber']
    custom_value_5 = row['fleetsize_lyft']
    custom_value_6 = row['fleetsize_cruise']
    custom_value_7 = row['fleetsize_flywheel']
    custom_value_8 = row['fleetsize_waymo']
    # convert the "lever_position_price" value from percent to decimal
    custom_value_1_decimal = float(custom_value_1.strip('%')) / 100
    custom_value_2_decimal = float(custom_value_2.strip('%')) / 100
    
    # get the filename for the current row
    filename = filenames[i]  
    
    # check if the filename already exists in the dictionary
    if filename not in custom_values:
        # create a new dictionary entry for the filename if it doesn't exist
        custom_values[filename] = []
        
    custom_value_dict = {'lever_position_price': custom_value_1_decimal, 'lever_position_fltsz': custom_value_2_decimal, 
                         'lever_n_fleets': custom_value_3, 'fleetsize_uber': custom_value_4, 'fleetsize_lyft': custom_value_5,
                         'fleetsize_cruise': custom_value_6,'fleetsize_flywheel': custom_value_7,'fleetsize_waymo': custom_value_8}
    
    # append the custom values to the existing list for the filename
    custom_values[filename].append(custom_value_dict)

In [18]:
%%time
dfs = []

for key in key_list:
    obj = s3.get_object(Bucket="beam-outputs", Key=key)
    sf_files = pd.read_csv(obj['Body'], compression = 'gzip', usecols = cols_to_use)
    
    filename = key.split('/')[-1].rstrip('.csv.gz')
    #filename = filename.split("_2019")[0]  # remove anything after the year 2019
    
    for custom_value_dict in custom_values[filename]:
        sf_files = sf_files.assign(**custom_value_dict)
    sf_files['year'] = 2018
    #append the dataframe to the list of dataframes
    dfs.append(sf_files)

<timed exec>:5: DtypeWarning: Columns (27,234,235) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:5: DtypeWarning: Columns (27,234,235) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 12min 26s
Wall time: 12min 27s


In [19]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [25]:
dfs[0]['socialCarbonCost'] = dfs[0]['emission_marginal']*185
dfs[0]['incomeInThousands'] = dfs[0]['income']/1000
dfs[0] = dfs[0][dfs[0]['incomeInThousands'].notna()]
person_income = pd.pivot_table(dfs[0], index=['IDMerged'], aggfunc={'incomeInThousands': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 

In [26]:
person_income.head()

,IDMerged,incomeInThousands
0,4,70.55
1,8,70.55
2,12,20.6
3,16,64.4
4,18,21.7


In [28]:
def generate_summary_table(df):
    df['socialCarbonCost'] = df['emission_marginal']*185
    df['incomeInThousands'] = df['income']/1000
    df = df[df['incomeInThousands'].notna()]
    person_income = pd.pivot_table(df, index=['IDMerged'], aggfunc={'incomeInThousands': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 
    person_income['incomeInThousands'] = person_income['incomeInThousands'].astype(float)

    # Add a column of income ranks
    twenty_one_ranks = person_income['incomeInThousands'].quantile([0, 0.048, 0.095, 0.143, 0.191, 0.239, 0.287, 0.335, 0.383, 0.431, 0.479,
                                                                0.527, 0.575, 0.623, 0.671, 0.719, 0.767, 0.815, 0.863, 0.911, 0.959,
                                                                1]).tolist()    
    # Add incomeInThousands twenty_one_ranks
    conditions  = [(person_income['incomeInThousands'] >= twenty_one_ranks[0]) & (person_income['incomeInThousands'] < twenty_one_ranks[1]), 
               (person_income['incomeInThousands'] >= twenty_one_ranks[1]) & (person_income['incomeInThousands'] < twenty_one_ranks[2]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[2]) & (person_income['incomeInThousands'] < twenty_one_ranks[3]),
               (person_income['incomeInThousands'] >= twenty_one_ranks[3]) & (person_income['incomeInThousands'] < twenty_one_ranks[4]), 
               (person_income['incomeInThousands'] >=  twenty_one_ranks[4]) & (person_income['incomeInThousands'] < twenty_one_ranks[5]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[5]) & (person_income['incomeInThousands'] < twenty_one_ranks[6]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[6]) & (person_income['incomeInThousands'] < twenty_one_ranks[7]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[7]) & (person_income['incomeInThousands'] < twenty_one_ranks[8]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[8]) & (person_income['incomeInThousands'] < twenty_one_ranks[9]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[9]) & (person_income['incomeInThousands'] <= twenty_one_ranks[10]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[10]) & (person_income['incomeInThousands'] <= twenty_one_ranks[11]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[11]) & (person_income['incomeInThousands'] <= twenty_one_ranks[12]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[12]) & (person_income['incomeInThousands'] <= twenty_one_ranks[13]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[13]) & (person_income['incomeInThousands'] <= twenty_one_ranks[14]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[14]) & (person_income['incomeInThousands'] <= twenty_one_ranks[15]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[15]) & (person_income['incomeInThousands'] <= twenty_one_ranks[16]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[16]) & (person_income['incomeInThousands'] <= twenty_one_ranks[17]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[17]) & (person_income['incomeInThousands'] <= twenty_one_ranks[18]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[18]) & (person_income['incomeInThousands'] <= twenty_one_ranks[19]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[19]) & (person_income['incomeInThousands'] <= twenty_one_ranks[20]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[20]) & (person_income['incomeInThousands'] <= twenty_one_ranks[21])]
    choices = [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    
    person_income['incRank'] = np.select(conditions, choices, default=None)
    
    df = pd.merge(left = df, right = person_income, how='left', on = ['IDMerged'], suffixes=('', '_drop'))
    
    df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)
    
    # Calculate the Potential INEXUS in dollar
    conditions1  = [(df['primary_purpose_x'] == 'work'),(df['primary_purpose_x'] == 'univ'),
               (df['primary_purpose_x'] == 'school'),(df['primary_purpose_x'] == 'escort'),
              (df['primary_purpose_x'] == 'shopping'),(df['primary_purpose_x'] == 'eatout'),
              (df['primary_purpose_x'] == 'othmaint'),(df['primary_purpose_x'] == 'social'),
              (df['primary_purpose_x'] == 'othdiscr'),(df['primary_purpose_x'] == 'atwork')]
                                                              
    choices1 = [-0.022, -0.0271, -0.0271, -0.0279, -0.0279, -0.0279, -0.0175, -0.0175, -0.0279, -0.0279]
    
    df['c_ivt'] = np.select(conditions1, choices1, default=np.nan)
    
    df['alpha'] = -0.6 *(df['c_ivt'])/(df['value_of_time'])
    
    df['Potential_INEXUS_in_dollar_2023'] = ((df['logsum_trip_Potential_INEXUS']/df['alpha'])/100)*1.75
    
    # Mean pivot table
    mean_inc_table = pd.pivot_table(df,index=['incRank'], aggfunc={'incomeInThousands': np.mean,
           'Potential_INEXUS_in_dollar_2023': np.mean,
            'duration_travelling': np.mean,
            'duration_door_to_door': np.mean,
            'duration_walking': np.mean,
            'duration_in_privateCar': np.mean,
            'duration_on_bike': np.mean,
            'duration_in_ridehail': np.mean,
            'duration_in_transit': np.mean,
            'waitTime_no_replanning': np.mean,
            'waitTime_replanning': np.mean,
            'distance_travelling': np.mean,
            'distance_walking': np.mean,
            'distance_bike': np.mean,
            'distance_ridehail': np.mean,
            'distance_privateCar': np.mean,
            'distance_transit': np.mean,
            'distance_mode_choice': np.mean,
            'replanning_status': np.mean,
            'fuel_marginal': np.mean,
            'emission_marginal': np.mean,
            'cost_BEAM': np.mean,
            'socialCarbonCost':np.mean,
           'lever_position_price':lambda x: ', '.join(set(x.dropna().astype(str))),
           'lever_position_fltsz':lambda x: ', '.join(set(x.dropna().astype(str))),
           'lever_n_fleets':lambda x: ', '.join(set(x.dropna().astype(str))),
            'fleetsize_uber':lambda x: ', '.join(set(x.dropna().astype(str))),
            'fleetsize_lyft':lambda x: ', '.join(set(x.dropna().astype(str))),
            'fleetsize_cruise':lambda x: ', '.join(set(x.dropna().astype(str))),
            'fleetsize_flywheel':lambda x: ', '.join(set(x.dropna().astype(str))),
            'fleetsize_waymo':lambda x: ', '.join(set(x.dropna().astype(str))),
           'IDMerged': 'nunique',
           'tripIndex': 'nunique'}).reset_index()
    # Median summary table
    median_inc_table = pd.pivot_table(df, index=['incRank'], aggfunc={'incomeInThousands': np.median,
           'Potential_INEXUS_in_dollar_2023': np.median,
            'duration_travelling': np.median,
            'duration_door_to_door': np.median,
            'duration_walking':np.median,
            'duration_in_privateCar': np.median,
            'duration_on_bike': np.median,
            'duration_in_ridehail': np.median,
            'duration_in_transit': np.median,
            'waitTime_no_replanning': np.median,
            'waitTime_replanning': np.median,
            'distance_travelling': np.median,
            'distance_walking': np.median,
            'distance_bike': np.median,
            'distance_ridehail': np.median,
            'distance_privateCar': np.median,
            'distance_transit': np.median,
            'distance_mode_choice': np.median,
            'replanning_status': np.median,
            'fuel_marginal': np.median,
            'emission_marginal': np.median,
            'cost_BEAM': np.median,
            'socialCarbonCost':np.median
           }).reset_index() 
   # Sum summary table
    sum_inc_table = pd.pivot_table(df, index=['incRank'], aggfunc={'duration_travelling': np.sum,
            'duration_door_to_door': np.sum,
            'duration_walking': np.sum,
            'duration_in_privateCar': np.sum,
            'duration_on_bike': np.sum,
            'duration_in_ridehail': np.sum,
            'duration_in_transit': np.sum,
            'waitTime_no_replanning': np.sum,
            'waitTime_replanning': np.sum,
            'distance_travelling': np.sum,
            'distance_walking': np.sum,
            'distance_bike': np.sum,
            'distance_ridehail': np.sum,
            'distance_privateCar': np.sum,
            'distance_transit': np.sum,
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'fuel_marginal': np.sum,
            'emission_marginal': np.sum,
            'cost_BEAM': np.sum,
            'socialCarbonCost':np.sum
           }).reset_index() 
            
    mode_counts = df.groupby(['incRank', 'mode_choice_actual_BEAM'])['mode_choice_actual_BEAM'].count().unstack().add_prefix('mode_').reset_index()
    sum_inc_table = sum_inc_table.merge(mode_counts, on='incRank').assign(mode_ridehail_total = lambda x: x['mode_ride_hail'] + x['mode_ride_hail_pooled'])
    sum_inc_table.columns = [col + '_sum' for col in sum_inc_table.columns]
    sum_inc_table = sum_inc_table.rename(columns={'incRank_sum': 'incRank'})
    mm_inc_table = pd.merge(mean_inc_table, median_inc_table, on='incRank', suffixes=('_mean', '_median'))
    summary_inc_table = pd.merge(mm_inc_table, sum_inc_table, on='incRank', how='left')
    summary_inc_table = summary_inc_table.rename(columns={'tripIndex': 'n_trips', 
                                                      'IDMerged': 'n_agents', 
                                                      'logsum_trip_Potential_INEXUS_median': 'Potential_INEXUS_median',
                                                      'logsum_trip_Potential_INEXUS_mean': 'Potential_INEXUS_mean',
                                                      'mode_ride_hail_sum': 'mode_ride_hail_solo_sum'})        
    
    summary_inc_table = summary_inc_table.iloc[:, :2].join(summary_inc_table.iloc[:, 2:].sort_index(axis=1)) 
    
    # shift column 'person' to first position
    third_column = summary_inc_table.pop('n_trips')
    
    # insert column using insert(position,column_name,first_column) function
    summary_inc_table.insert(2, 'n_trips', third_column)
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_no_replanning = df.groupby(['mode_choice_actual_BEAM', 'incRank'])['waitTime_no_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_no_replanning, index=['incRank'], columns=['mode_choice_actual_BEAM'],
    values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_no_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='incRank', how='left')
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_replanning = df.groupby(['mode_choice_planned_BEAM', 'incRank'])['waitTime_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_replanning, index=['incRank'], columns=['mode_choice_planned_BEAM'], values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='incRank', how='left')
    return summary_inc_table

In [29]:
%%time
dfs_summary = []
for df in dfs:
    df_summary = generate_summary_table(df)
    dfs_summary.append(df_summary)

CPU times: total: 9min 4s
Wall time: 9min 4s


In [30]:
%%time
sf_stacked = pd.concat(dfs_summary)

CPU times: total: 0 ns
Wall time: 1.96 ms


In [32]:
sf_stacked

,incRank,n_agents,n_trips,Potential_INEXUS_in_dollar_2023_mean,Potential_INEXUS_in_dollar_2023_median,cost_BEAM_mean,cost_BEAM_median,cost_BEAM_sum,distance_bike_mean,distance_bike_median,distance_bike_sum,distance_mode_choice_mean,distance_mode_choice_median,distance_mode_choice_sum,distance_privateCar_mean,distance_privateCar_median,distance_privateCar_sum,distance_ridehail_mean,distance_ridehail_median,distance_ridehail_sum,distance_transit_mean,distance_transit_median,distance_transit_sum,distance_travelling_mean,distance_travelling_median,distance_travelling_sum,distance_walking_mean,distance_walking_median,distance_walking_sum,duration_door_to_door_mean,duration_door_to_door_median,duration_door_to_door_sum,duration_in_privateCar_mean,duration_in_privateCar_median,duration_in_privateCar_sum,duration_in_ridehail_mean,duration_in_ridehail_median,duration_in_ridehail_sum,duration_in_transit_mean,duration_in_transit_median,duration_in_transit_sum,duration_on_bike_mean,duration_on_bike_median,duration_on_bike_sum,duration_travelling_mean,duration_travelling_median,duration_travelling_sum,duration_walking_mean,duration_walking_median,duration_walking_sum,emission_marginal_mean,emission_marginal_median,emission_marginal_sum,fleetsize_cruise,fleetsize_flywheel,fleetsize_lyft,fleetsize_uber,fleetsize_waymo,fuel_marginal_mean,fuel_marginal_median,fuel_marginal_sum,incomeInThousands_mean,incomeInThousands_median,lever_n_fleets,lever_position_fltsz,lever_position_price,mode_bike_sum,mode_bike_transit_sum,mode_car_hov2_sum,mode_car_hov3_sum,mode_car_sum,mode_drive_transit_sum,mode_hov2_teleportation_sum,mode_hov3_teleportation_sum,mode_ride_hail_pooled_sum,mode_ride_hail_solo_sum,mode_ridehail_total_sum,mode_walk_sum,mode_walk_transit_sum,replanning_status_mean,replanning_status_median,replanning_status_sum,socialCarbonCost_mean,socialCarbonCost_median,socialCarbonCost_sum,waitTime_no_replanning_mean,waitTime_no_replanning_median,waitTime_no_replanning_sum,waitTime_replanning_mean,waitTime_replanning_median,waitTime_replanning_sum,waitTime_no_replanning_bike_mean,waitTime_no_replanning_bike_transit_mean,waitTime_no_replanning_car_mean,waitTime_no_replanning_car_hov2_mean,waitTime_no_replanning_car_hov3_mean,waitTime_no_replanning_drive_transit_mean,waitTime_no_replanning_hov2_teleportation_mean,waitTime_no_replanning_hov3_teleportation_mean,waitTime_no_replanning_ride_hail_mean,waitTime_no_replanning_ride_hail_pooled_mean,waitTime_no_replanning_walk_mean,waitTime_no_replanning_walk_transit_mean,waitTime_no_replanning_bike_median,waitTime_no_replanning_bike_transit_median,waitTime_no_replanning_car_median,waitTime_no_replanning_car_hov2_median,waitTime_no_replanning_car_hov3_median,waitTime_no_replanning_drive_transit_median,waitTime_no_replanning_hov2_teleportation_median,waitTime_no_replanning_hov3_teleportation_median,waitTime_no_replanning_ride_hail_median,waitTime_no_replanning_ride_hail_pooled_median,waitTime_no_replanning_walk_median,waitTime_no_replanning_walk_transit_median,waitTime_replanning_bike_mean,waitTime_replanning_bike_transit_mean,waitTime_replanning_car_mean,waitTime_replanning_car_hov2_mean,waitTime_replanning_car_hov3_mean,waitTime_replanning_drive_transit_mean,waitTime_replanning_hov2_teleportation_mean,waitTime_replanning_hov3_teleportation_mean,waitTime_replanning_ride_hail_mean,waitTime_replanning_ride_hail_pooled_mean,waitTime_replanning_walk_mean,waitTime_replanning_walk_transit_mean,waitTime_replanning_bike_median,waitTime_replanning_bike_transit_median,waitTime_replanning_car_median,waitTime_replanning_car_hov2_median,waitTime_replanning_car_hov3_median,waitTime_replanning_drive_transit_median,waitTime_replanning_hov2_teleportation_median,waitTime_replanning_hov3_teleportation_median,waitTime_replanning_ride_hail_median,waitTime_replanning_ride_hail_pooled_median,waitTime_replanning_walk_median,waitTime_replanning_walk_transit_median
0,0,91811,319870,-2.515778,-1.366920,0.877347,0.341667,280636.854563,73.

In [16]:
sf_stacked.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_rh_fltsz_price_incRank_3.csv', index=False)

In [21]:
%%time
stacked_rh_path = "https://beam-core-act.s3.amazonaws.com/deepDive/CleanData/SanFrancisco/Stacked/"
sf_stacked_12 = pd.read_csv(stacked_rh_path + 'sf_2018_stacked_rh_fltsz_price_incRank_1_2.csv')

CPU times: total: 15.6 ms
Wall time: 77.1 ms


In [24]:
frames = [sf_stacked_12, sf_stacked]
sf_stacked123 = pd.concat(frames)

In [25]:
sf_stacked123.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_rh_fltsz_price_incRank_1_2_3.csv', index=False)